# N Body Problem Simulator
 By N. Ricker

In [5]:
# Import Modules
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [6]:
# Constants
G = 6.67408e-11 # m^3 kg^-1 s^-2

In [46]:
# Body class
class Body:
    def __init__(self, name='x', mass=1, radius=0, position=[0,0,0], velocity=[0,0,0]):
        self.name = name
        self.mass = mass
        self.radius = radius
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.state = np.concatenate((self.position, self.velocity))

In [8]:
# System class
class System:
    def __init__(self):
        self.bodies = []
        self.positions = np.array([])
        self.velocities = np.array([])
        self.stateVector = np.concatenate((self.positions, self.velocities))

    def add_body(self, body):
        self.bodies.append(body)
        self.positions = np.append(self.positions, body.position)
        self.velocities = np.append(self.velocities, body.velocity)
        self.stateVector = np.concatenate((self.positions, self.velocities))

In [47]:
# Differential equation
def diff_eq(stateVector, t, system):
    # Initialize
    dydt = np.zeros_like(stateVector)
    N = len(system.bodies)
    dydt[:3*N] = stateVector[3*N:]
    
    for i, body in enumerate(system.bodies):
        ai = np.zeros(3)
        for j, pbody in enumerate(system.bodies):
            if i==j:
                continue
            posBody = stateVector[3*i:3*(i+1)]
            posPBody = stateVector[3*j:3*(j+1)]
            r = posPBody - posBody
            ai += G*pbody.mass*r/np.linalg.norm(r)**3
        dydt[3*N + 3*i:3*N + 3*(i+1)] = ai
    return dydt

In [56]:
# Solve differential equation for system based on solar system
t = np.linspace(0, 365*24*3600, 100000)
system = System()
system.add_body(Body("Sun", 1.989e30, 0, [0,0,0], [0,0,0]))                    # Sun
system.add_body(Body("Mercury", 3.301e23, 0, [5.791e10,0,0], [0,4.787e4,0]))     # Mercury
#system.add_body(Body("Venus", 4.867e24, 0, [1.082e11,0,0], [0,3.502e4,0]))       # Venus
system.add_body(Body("Earth", 5.972e24, 0, [1.496e11,0,0], [0,2.978e4,0]))       # Earth
system.add_body(Body("Moon", 7.34767309e22, 0, [1.496e11+3.844e8,0,0], [0,2.978e4+1.022e3,0]))       # Moon
#system.add_body(Body("Mars", 6.417e23, 0, [2.279e11,0,0], [0,2.407e4,0]))        # Mars
#system.add_body(Body("Jupiter", 1.899e27, 0, [7.785e11,0,0], [0,1.307e4,0]))     # Jupiter
#system.add_body(Body("Saturn", 5.685e26, 0, [1.433e12,0,0], [0,9.69e3,0]))       # Saturn
#system.add_body(Body("Uranus", 8.682e25, 0, [2.877e12,0,0], [0,6.81e3,0]))       # Uranus


sol = odeint(diff_eq, system.stateVector, t, args=(system,))

In [58]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111)

for i, body in enumerate(system.bodies):
    ax.plot(sol[::10,3*i], sol[::10,3*i+1], label=body.name)

ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.axis('equal')
ax.legend()
plt.show()
    